## Decision Trees (Classification)

All decision trees work in basically the same way. Imagine we have data matrix X with $n$ samples and $d$ features. $X \in \mathbb{R}^{n\times d}$

1. Pick a feature based on some splitting criterion
2. Divide dataset based on feature, each branch has that data
3. Recurse


In [108]:
import numpy as np
import pandas as pd
from typing import List
from abc import *

### Datasets

In [435]:
train = pd.read_csv("datasets/titanic_train.csv")
test = pd.read_csv("datasets/titanic_test.csv")

X_train, y_train = train.drop(["Survived", "Name", "Ticket", "Cabin", "PassengerId", "Parch"], axis=1), train['Survived']
#X_train['Age'].hist(bins=10)
X_train['SibSp'].value_counts()


SibSp
0    608
1    209
2     28
4     18
3     16
8      7
5      5
Name: count, dtype: int64

In [321]:
def quantile_threshold(data, N):
    thresholds = data.quantile([i / N for i in range(1, N)])
    def map_func(x):
        for i, val in enumerate(thresholds):
            if x <= val:
                return i
        return N-1
    return data.map(map_func)
        

def categorize_titanic(X):
    df = X.copy()

    for column in df.columns:
        if df[column].isna().any():
            majority_value = df[column].mode().iloc[0]  # Get the most frequent value
            df[column] = df[column].fillna(majority_value)
    
    df['Sex'] = df['Sex'].map({'male': 1, 'female': 0})
    df['Embarked'] = df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
    df['Pclass'] = df['Pclass'].map({1: 0, 2: 1, 3: 2})
    df['Fare'] = quantile_threshold(df['Fare'], 3)
    df['Age'] = quantile_threshold(df['Age'], 2)
    df['SibSp'] = df['SibSp'].map({0: 0, 1: 1, 2: 2, 3: 2, 4: 2, 5: 2, 6: 2, 8: 2})

    return df


In [339]:
X_train, y_train = train.drop(["Survived", "Name", "Ticket", "Cabin", "PassengerId", "Parch"], axis=1), train['Survived']
X_train = categorize_titanic(X_train)

X_train.isna().any()


Pclass      False
Sex         False
Age         False
SibSp       False
Fare        False
Embarked    False
dtype: bool

In [439]:
X_test = test.drop([ "Name", "Ticket", "Cabin", "PassengerId", "Parch"], axis=1)
X_test = categorize_titanic(X_test)


### Splitting Metric

In [ ]:
variable_value_map = {
    'Sex': {0: 'female', 1: 'male'},
    'Embarked': {0: 'Southampton', 1: 'Cherbourg', 2: 'Queenstown'},
    'Pclass': {0: "first", 1: "second", 2: "third"},
    'Fare': {0: 'low', 1: 'medium', 2: 'high'},
    'Age': {0: 'young', 1: 'old'},
    'SibSp': {0: '0', 1: '1', 2: '2+'}
}

def get_varcode(feature, value):
    vals = variable_value_map[feature]
    for key, val in vals.items():
        if isinstance(value, str) and val == value:
            return key
        elif isinstance(value, int) and key == value:
            return val
    return None

get_varcode('Sex', 'male')

1

In [120]:

def entropy(y):
    probs = y.value_counts(normalize=True)
    return -1 * np.sum(probs * np.log2(probs + 1e-9))

def classification_error(y):
    return 1 - y.value_counts().max() / y.count()

def gini_index(y):
    N = y.count()
    return 1 - y.value_counts().pow(2).sum() / N**2

entropy(pd.Series([2,1,0,1]))

np.float64(1.4999999956719148)

### Tree Structure

In [119]:
X_train['Sex'].size

891

In [324]:
uncertainty_measures = {
	'entropy': entropy,
	'classification_error': classification_error,
	'gini_index': gini_index
}

class DecisionTree:
	def __init__(self, data, labels, uncertainty_measure='entropy'):
		self.X = data
		self.y = labels
		self.features = list(data.columns)
		self.N = data.shape[0]
		self.uncertainty_measure = uncertainty_measure
		self.uncertainty_func = uncertainty_measures[uncertainty_measure]
		self.splitting_feature = None
		self.children = []
		self.is_leaf = (len(self.features) == 1) or self.N < 5
	
	@abstractmethod
	def _splits(self, feature):
		pass

	def Information_Gain(self, feature):
		data = self.X[feature]
		G = self.uncertainty_func(self.y)
		for split in self._splits(feature):
			G -= split.size / self.N * self.uncertainty_func(split)
		return G




In [333]:
y_counts = y_train.value_counts(normalize=True)
y_counts.idxmax().item(), y_counts[y_counts.idxmax().item()]

(0, np.float64(0.6161616161616161))

In [428]:

class CategoricalDecisionTree(DecisionTree):
	def __init__(self, data, labels, uncertainty_measure='entropy'):
		super().__init__(data, labels, uncertainty_measure)
		self.children = {}
		#self.gains = {feature : self.Information_Gain(feature) for feature in self.features}
		if(self.N > 0):
			self.fit()
	
	def _splits(self, feature):
		res = []
		for i in self.X[feature].unique():
			res.append(self.y[self.X[feature] == i].reset_index(drop=True))
		return res
	
	def select_splitting_feature(self):
		max_gain = 0
		for feature in self.features:
			gain = self.Information_Gain(feature)
			if gain > max_gain:
				max_gain = gain
				self.splitting_feature = feature
		if(self.splitting_feature is None):
			gains = {feature : self.Information_Gain(feature) for feature in self.features}
			print("No gain acheived, need to diagnose ", gains)
			self.weird = True
		return max_gain

	
	def fit(self, debug=False):
		y_probs = self.y.value_counts(normalize=False)
		if(debug):
			print(f"\nfitting node with y_probs={dict(y_probs)} samples on features {self.features}")
		majority_yval = y_probs.idxmax().item()
		if(y_probs[majority_yval] / self.N > 0.85 or self.N < 5):
			if(debug):
				print("Enforcing leaf node due to purity")
			self.is_leaf = True
		
		if self.is_leaf:
			self.predictions = {}
			self.splitting_feature = self.features[0]
			for xval in variable_value_map[self.splitting_feature].keys():
				y = self.y[self.X[self.splitting_feature] == xval]
				if(y.size > 1):
					self.predictions[int(xval)] = y.value_counts().idxmax().item()
				else:
					self.predictions[int(xval)] = majority_yval
			if(debug):
				print(f"Leaf node on {self.splitting_feature}, predictions: {self.predictions}")
			return
		
		max_gain = self.select_splitting_feature()
		if(debug):
			print(f"Choosing to split on {self.splitting_feature} with gain {max_gain}")
		
		for xval in self.X[self.splitting_feature].unique():
			bool_mask = self.X[self.splitting_feature] == xval
			sample = self.X[bool_mask].drop(columns=[self.splitting_feature]).reset_index(drop=True)
			new_label = self.y[bool_mask].reset_index(drop=True)
			try:
				self.children[int(xval)] = CategoricalDecisionTree(sample, new_label, self.uncertainty_measure)
			except Exception as e:
				print(f"Error fitting child with x value {xval}, {e}")
				self.weird = True

	def predict_row(self, row, indent_level=0, verbose=False):
		rval = int(row[self.splitting_feature])
		if not self.is_leaf:
			space = "   " * indent_level
			if verbose:
				print(f"{space} -->{self.splitting_feature} is {variable_value_map[self.splitting_feature][rval]} ({rval})")
			return self.children[rval].predict_row(row, indent_level + 1, verbose)
		pred = self.predictions[rval]
		if verbose:
			print(f"And so finally, because {self.splitting_feature} is {variable_value_map[self.splitting_feature][rval]} ({rval}), the prediction is {pred}")
		return pred

	def predict(self, data):
		preds = []
		for i, row in data.iterrows():
			preds.append(self.predict_row(row, verbose=False))
		return np.array(preds)


In [429]:
tree = CategoricalDecisionTree(X_train, y_train)
tree.fit()

In [417]:
row = X_train.iloc[6]
y_train.iloc[6].item(), row

(0,
 Pclass      0
 Sex         1
 Age         1
 SibSp       0
 Fare        2
 Embarked    0
 Name: 6, dtype: int64)

In [418]:
tree.predict_row(row, verbose=True)

 -->Sex is male (1)
    -->Fare is high (2)
       -->SibSp is 0 (0)
          -->Pclass is first (0)
             -->Age is old (1)
And so finally, because Embarked is Southampton (0), the prediction is 0


0

In [432]:
(tree.predict(X_train) == y_train).mean()

np.float64(0.8383838383838383)

In [440]:
tree.predict(X_test)

array([0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [386]:
curr = tree
rval = int(row[curr.splitting_feature])
print(f"--> {curr.splitting_feature} is {variable_value_map[curr.splitting_feature][rval]} ({rval})")
curr.children

--> Sex is male (1)


{1: <__main__.CategoricalDecisionTree at 0x2925cc890>,
 0: <__main__.CategoricalDecisionTree at 0x2912c8890>}

In [392]:
curr = curr.children[rval]
rval = int(row[curr.splitting_feature])
print(f"--> {curr.splitting_feature} is {variable_value_map[curr.splitting_feature][rval]} ({rval})")
curr.is_leaf, curr.children

--> Embarked is Southampton (0)


(True, {})

In [394]:
curr.predictions, curr.splitting_feature

({0: 0, 1: 0, 2: 0}, 'Embarked')